In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

category_links = [
    "https://voxiitk.com/category/all-about-iitk/",
    "https://voxiitk.com/category/editorials/",
    "https://voxiitk.com/category/flagship-series/",
    "https://voxiitk.com/category/reports-and-investigations/",
    "https://voxiitk.com/category/surveys/",
    "https://voxiitk.com/category/beyond-iitk/"
]

CHROMEDRIVER_PATH = r"C:\Users\MANTHAN KHETADE\Desktop\chromedriver-win64\chromedriver-win64\chromedriver.exe"

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=chrome_options)

def get_article_links_from_category(category_url):
    article_links = set()
    page_num = 1

    while True:
        page_url = f"{category_url.rstrip('/')}/page/{page_num}/"
        
        driver.get(page_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'lxml')
        article_anchors = soup.find_all('a', href=True)

        page_article_links = set()

        for link in article_anchors:
            href = link['href']
            if (href.startswith("https://voxiitk.com/") and
                'category/' not in href and
                'tag/' not in href and
                'page/' not in href and
                '#' not in href and
                href.endswith('/')):
                page_article_links.add(href)

        if len(page_article_links) == 0:
            break

        article_links.update(page_article_links)

        if len(page_article_links) < 10:
            break

        page_num += 1
        time.sleep(1)
    return article_links

def scrape_article_content(article_url):
    driver.get(article_url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    paragraphs = soup.find_all('p')

    content = "\n".join(p.get_text().strip() for p in paragraphs if p.get_text().strip())

    return content

def scrape_all_articles(category_links):
    all_articles_content = ""

    for category_url in category_links:
        article_links = get_article_links_from_category(category_url)
        for article_url in article_links:
            try:
                content = scrape_article_content(article_url)
                all_articles_content += f"\n\n---\n\n{article_url}\n\n{content}\n\n"
                time.sleep(1)
            except Exception as e:
                print(f"Failed to scrape {article_url}: {e}")
                continue

    return all_articles_content

if __name__ == "__main__":
    scraped_text = scrape_all_articles(category_links)

    with open("iitk_vox_articles_finaa.txt", "w", encoding="utf-8") as f:
        f.write(scraped_text)

    driver.quit()
